## Introduction
Say we have a rnn and we want it to decide on its own when to stop.

One approach is to consider the last layer at each timestep. We can do this by seeing whether a halting unit passed some threshold. Also, we can make it stochastic and define the probability of a halt using a binomial distribution given by the value of the halting unit. Another option is to measure its security on the output for the task. We can do this by executing the neural network multiple times and observing the variance of the result (low variance means security.) 

Other idea is to interpret the halting unit at one timestep as how much we should consider the output at that timestep. This generalizes the previous idea because the nn can output always 0 in the halting unit and then output 1.

{how do self-delimiting neural networks train the halting unit?}

## 2. ACT
Recall that for RNNs we have two dimensions. First, we have the time dimension to process long sequences applying the same LSTM transformation. Second: we have the processing dimension where we may want multiple layers to process one input (this is super common in feedforward nns.)

<img src="https://slideplayer.com/slide/14889574/91/images/7/Deep+RNN+RNNs+with+multiple+hidden+layers+y1+y2+y3+y4+y5+y6+x1+x2+x3.jpg" style="width: 200px">

The modification from ACT is to make variable the length of the processing dimension. In principle, this only applies to rnns. However, another way to do this is to have a halting unit at each layer in a feedforward nn. So that we halt execution whenever one halting unit passed some threshold (other option: the accumulated value of the halting unit is greater than threshold.) 

Given that we are in time t, for each processing step n we have:
$$
s_t^n = S(s_{t-1}, x_t^1) \ if\  n = 1 \\
= S(s_t^{n-1}, x_t^n)\ otherwise \\
y_t^n = W_ys_t^n + b_y
$$

Where $x_t^n = x_t + \gamma_{n,1}$. $\gamma_{l, k}$ is a binary flag that is one whenever l == k. Without this flag, a repeated input (ie $x_{t-1} == x_t$) would be the same for the rnn as two consecutive computations ($x_t^{n - 1} == x_t^n$) (because the state s would be the same when adding one processing step as when starting a new time step.) The computation is the same for each processing step n, but we could also be using different matrices for each time step.

For each time t and for each processing step n we produce three vectors:
$$y_t^n, s_t^n, h_t^n$$

$h_t^n$ stores the halting variable. If $\sum_n^K h_t^n > 1 - \epsilon,$ then we halt execution and we set N(t) = K. Then, we copy $h_t^n$ into $p_t^n$ for each n except for $n = N(t)$ (the last value of n.) To make $\sum_n^{N(t)} h_t^n$ equals to $1,$ we set $R(t) = p_t^{N(t)} = 1 - \sum_n^{N(t)-1} h_t^n$

Then, the output vector in the processing dimension is as follows
$$s_t = \sum_n^{N(t)} p_t^n s_t^n$$
$$y_t = \sum_n^{N(t)} p_t^n y_t^n$$

### 2.1
We'd want to penalize using too much time to think. Thus, we define the ponder cost at timestep t as the sum of the processing steps used (N(t)) and the remaining budget after the last processing step (R(t))
$$p_t = N(t) + R(t)$$
$$P = \sum_t p_t$$
$$\hat L = L + \tau p$$
We can't minimize N(t), but by minimizing R(t) we are making each processing step bigger and thus indirectly minimizing N(t). 

### 2.2
$\newcommand{\pd}[2]{\frac{\partial#1}{\partial#2}}$
$$\pd{P}{h_t^n} = 0\ if\ n = N(t)\  else\  -1$$

Finding this limit in its most general
form would be equivalent to determining the Kolmogorov complexity of the data (and hence solving
the halting problem) [21]. why?
Hogwild!

## 3
Penalizing less the ponder time allows for more ponder time, which in turn allows a lower sequence error rate at the cost of more processing.

# Terms
Sequence error rate: times there was at least one error in the sequence.
Mean-field vector: weighted average